In [1]:
import sys
sys.path.insert(1, '../../scripts/')

from s3_support import *

In [2]:
q = "select id, org, form, message, created from syslog_logs where message like '%Smart Amounts%'"
df = redshift_query_read(q, schema='production')

In [3]:
df['enabled'] = df['message'].str.contains('Enabled').fillna(False)
df['disabled'] = df['message'].str.contains('Disabled').fillna(False)

In [4]:
df[['enabled', 'disabled']].value_counts()

enabled  disabled
True     False       1369
False    True         954
dtype: int64

In [12]:
df[['form', 'enabled', 'disabled', 'created']].tail(3)

,form,enabled,disabled,created
69,449054,True,False,2022-11-14 14:19:34
70,443689,True,False,2022-12-06 11:21:52
71,444174,True,False,2022-12-10 14:50:37


In [5]:
forms_enabled = df[(df['enabled'])&(df['created']<='2024-12-03')]['form'].tolist()
forms_disabled = df[(df['disabled'])&(df['created']<'2024-12-03')]['form'].tolist()

In [6]:
len(forms_enabled), len(forms_disabled)

(1369, 954)

In [7]:
active_forms = [f for f in forms_enabled if f not in forms_disabled]
len(active_forms)

727

In [8]:
activation_sums = df.groupby('form')[['enabled', 'disabled']].sum().reset_index()
active_forms = activation_sums[(activation_sums['enabled']>activation_sums['disabled'])&(activation_sums['form'].isin(forms_enabled))]['form'].tolist()
len(active_forms)

784

# Giving tuesday 

In [24]:
q = '''select
            form,
            count(id) as count,
            sum(amount) as volume
        from transactions
        where
            status='A' and
            date='2024-12-03' 
        group by form'''
trans = redshift_query_read(q, schema='production')

In [25]:
trans['mean_trans_amount'] = trans['volume'] / trans['count']
trans['smart_amounts'] = trans['form'].isin(active_forms)

In [29]:
aggs = {
    'mean': 'count',
    'mean': 'volume',
    'mean': 'mean_trans_amount'
}

trans.groupby('smart_amounts')[['count', 'volume', 'mean_trans_amount']].agg(['mean', 'median']).reset_index().to_markdown()

"|    | ('smart_amounts', '')   |   ('count', 'mean') |   ('count', 'median') |   ('volume', 'mean') |   ('volume', 'median') |   ('mean_trans_amount', 'mean') |   ('mean_trans_amount', 'median') |\n|---:|:------------------------|--------------------:|----------------------:|---------------------:|-----------------------:|--------------------------------:|----------------------------------:|\n|  0 | False                   |             9.76825 |                     3 |              1839.42 |                 312.55 |                         234.079 |                           100.032 |\n|  1 | True                    |             9.16554 |                     3 |              1914.54 |                 469    |                         185.247 |                           107.655 |"

In [28]:
_df = trans[trans['form'].isin(active_forms)]

print("smart amounts forms (giving tuesday 2024):")
print("-"*40)
print("{:,} forms using smart amounts active on GT 2024".format(len(_df)))

trans.groupby('smart_amounts')[['count', 'volume', 'mean_trans_amount']].agg(['mean', 'median']).reset_index()

smart amounts forms (giving tuesday 2024):
----------------------------------------
296 forms using smart amounts active on GT 2024


smart_amounts     count              volume         mean_trans_amount  \
                     mean median         mean  median              mean   
0         False  9.768255    3.0  1839.423002  312.55        234.078599   
1          True  9.165541    3.0  1914.541385  469.00        185.247095   

               
       median  
0  100.032156  
1  107.655000

# checking production forms

In [47]:
production_forms = [41, 939268, 953991, 995353, 997346, 
                    997565, 997774, 997868]

df[df['form'].isin(production_forms)]

,id,org,form,message,created,enabled,disabled


In [49]:
df['form'].unique()

array([444866, 444339,    833, 443294,   1229, 449054, 443495, 443689,
       444174, 447126, 449435, 447063, 448312,     67,    427, 449490,
       448952, 441929,    821, 428045, 446218, 445312, 444852, 445128,
       446754,  34809,  39922, 443239,    826, 442437, 449520, 448326,
       444778, 447209, 447894, 445362])

In [78]:
q = '''select
            id,
            form,
            amount
        from transactions
        where 
            date='2022-11-29' and
            status='A' and
            recurring=0 and
            ({})'''.format(" or ".join(["form={}".format(f) for f in production_forms]))
pro_trans = redshift_query_read(q, schema='production')
print(len(pro_trans))

198


In [76]:
print("Count: {}".format(len(pro_trans)))
print("Forms: {}".format(len(pro_trans['form'].unique())))
print("Max amount: ${:,.2f}".format(pro_trans['amount'].max()))
print("Min amount: ${:,.2f}".format(pro_trans['amount'].min()))
print("Mean amount: ${:.2f}".format(pro_trans['amount'].mean()))
print("Median amount: ${:.2f}".format(pro_trans['amount'].median()))

Count: 198
Forms: 7
Max amount: $3,500.00
Min amount: $5.00
Mean amount: $111.00
Median amount: $53.00


In [80]:
q = '''select
            id,
            amount
        from transactions
        where
            date='2022-11-29' and
            status='A' and
            recurring=0 and
            ({})'''.format(" or ".join(["form!={}".format(f) for f in production_forms]))
ex_trans = redshift_query_read(q, schema='production')

In [83]:
print("Count: {}".format(len(ex_trans)))
print("Max amount: ${:,.2f}".format(ex_trans['amount'].max()))
print("Min amount: ${:,.2f}".format(ex_trans['amount'].min()))
print("All mean amount: ${:.2f}".format(ex_trans['amount'].mean()))
print("All median amount: ${:.2f}".format(ex_trans['amount'].median()))

Count: 31379
Max amount: $50,000.00
Min amount: $0.00
All mean amount: $182.80
All median amount: $52.50
